In [23]:
using LazySets, BenchmarkTools, Test, StaticArrays, LinearAlgebra

In [90]:
function _is_intersection_empty_v1(Z::Zonotope{N},
                                H::Hyperplane{N}
                              ) where {N<:Real}

    v = H.b - dot(H.a, Z.center)
    p = ngens(Z)
    if p == 0
        abs_sum = zero(N)
    else
        abs_sum = sum(abs, transpose(H.a) * Z.generators)        
        #abs_sum = sum(abs(dot(H.a, Z.generators[:, i])) for i = 1:ngens(Z))
    end
    return v < -abs_sum || v > abs_sum
end

function _is_intersection_empty_v2(Z::Zonotope{N},
                                   H::Hyperplane{N}
                                   ) where {N<:Real}

    c, G = Z.center, Z.generators
    v = H.b - dot(H.a, c)

    n, p = size(G)
    if p == 0
        abs_sum = zero(N)
    else
        #abs_sum = sum(abs, transpose(H.a) * Z.generators)        
        #abs_sum = sum(abs(dot(H.a, Z.generators[:, i])) for i = 1:ngens(Z))
        
        abs_sum = zero(N)
        @avx for i in 1:n
            aux = zero(N)
            for j in 1:p
                aux += H.a[i] * G[j, i]
            end
            abs_sum += abs(aux)
        end
    end
    #return v < -abs_sum || v > abs_sum
    return !_geq(v, -abs_sum) || !_leq(v, abs_sum)
end

using LazySets: _leq, _geq
function _is_intersection_empty_v3(Z::Zonotope{N},
                                   H::Hyperplane{N}
                                   ) where {N<:Real}

    c, G = Z.center, Z.generators
    v = H.b - dot(H.a, c)

    n, p = size(G)
    abs_sum = zero(N)
    if p > 0
        @inbounds for i in 1:n
            aux = zero(N)
            @simd for j in 1:p
                aux += H.a[i] * G[j, i]
            end
            abs_sum += abs(aux)
        end
    end
    return !_geq(v, -abs_sum) || !_leq(v, abs_sum)
end

_is_intersection_empty_v3 (generic function with 1 method)

## Robustness

In [14]:
using Plots

In [81]:
p = Float64(π)
G = Hyperplane([1.0, 0.0], p + 1e-12) # x = π
Z = convert(Zonotope, Hyperrectangle(low=[p-1, p-1], high=[p, p]))
plot(Z, lab="Z")
plot!(G, lab="G", lw=10.0)

@show isdisjoint(Z, G)
@show _is_intersection_empty_v3(Z, G)

isdisjoint(Z, G) = true
_is_intersection_empty_v3(Z, G) = true


true

In [78]:
LazySets._rtol(Float64)

1.4901161193847656e-8

In [85]:
LazySets.set_rtol(Float64, 1e-9)

1.0e-9

In [86]:
p = Float64(π)
G = Hyperplane([1.0, 0.0], p - 1e-10) # x = π
Z = convert(Zonotope, Hyperrectangle(low=[p, p], high=[p+1, p+1]))
plot(Z, lab="Z")
plot!(G, lab="G", lw=10.0)

@show isdisjoint(Z, G)
@show _is_intersection_empty_v3(Z, G)

isdisjoint(Z, G) = true
_is_intersection_empty_v3(Z, G) = false


false

## Dimension 2

In [87]:
n = 2

Z = rand(Zonotope, num_generators=5*n, dim=n)
G = rand(Hyperplane, dim=n)

@btime is_intersection_empty($Z, $G)

# static
p = size(Z.generators, 2);
Zs = Zonotope(SVector{n}(Z.center), SMatrix{n, p, Float64}(Z.generators));
Gs = Hyperplane(SVector{n}(G.a), G.b);

@btime is_intersection_empty($Zs, $Gs)

# new function
@btime _is_intersection_empty_v1($Z, $G)
@btime _is_intersection_empty_v1($Zs, $Gs)

# new function
@btime _is_intersection_empty_v2($Z, $G)
@btime _is_intersection_empty_v2($Zs, $Gs)

# final function
@btime _is_intersection_empty_v3($Z, $G)
@btime _is_intersection_empty_v3($Zs, $Gs)

  471.015 ns (10 allocations: 960 bytes)
  23.803 ns (0 allocations: 0 bytes)
  121.679 ns (1 allocation: 160 bytes)
  6.937 ns (0 allocations: 0 bytes)
  29.834 ns (0 allocations: 0 bytes)
  5.172 ns (0 allocations: 0 bytes)
  29.341 ns (0 allocations: 0 bytes)
  4.917 ns (0 allocations: 0 bytes)


false

sin `@simd`

```julia
  472.474 ns (10 allocations: 960 bytes)
  22.652 ns (0 allocations: 0 bytes)
  121.649 ns (1 allocation: 160 bytes)
  6.956 ns (0 allocations: 0 bytes)
  30.862 ns (0 allocations: 0 bytes)
  5.666 ns (0 allocations: 0 bytes)
```

con `@simd` en el loop de afuera

```julia
  470.735 ns (10 allocations: 960 bytes)
  23.792 ns (0 allocations: 0 bytes)
  122.094 ns (1 allocation: 160 bytes)
  6.952 ns (0 allocations: 0 bytes)
  28.381 ns (0 allocations: 0 bytes)
  5.670 ns (0 allocations: 0 bytes)
```

con `@simd` en el loop de adentro:

```julia
  471.862 ns (10 allocations: 960 bytes)
  23.777 ns (0 allocations: 0 bytes)
  120.735 ns (1 allocation: 160 bytes)
  6.953 ns (0 allocations: 0 bytes)
  28.559 ns (0 allocations: 0 bytes)
  4.151 ns (0 allocations: 0 bytes)
true
```

con `@avx`


```julia
  471.444 ns (10 allocations: 960 bytes)
  23.797 ns (0 allocations: 0 bytes)
  121.364 ns (1 allocation: 160 bytes)
  6.967 ns (0 allocations: 0 bytes)
  23.832 ns (0 allocations: 0 bytes)
  4.653 ns (0 allocations: 0 bytes)
```

final  (v3)

```julia
  471.015 ns (10 allocations: 960 bytes)
  23.803 ns (0 allocations: 0 bytes)
  121.679 ns (1 allocation: 160 bytes)
  6.937 ns (0 allocations: 0 bytes)
  29.834 ns (0 allocations: 0 bytes)
  5.172 ns (0 allocations: 0 bytes)
  29.341 ns (0 allocations: 0 bytes)
  4.917 ns (0 allocations: 0 bytes)
```

## Dimension 10

In [91]:
n = 10

Z = rand(Zonotope, num_generators=5*n, dim=n)
G = rand(Hyperplane, dim=n)

@btime is_intersection_empty($Z, $G)

# static
p = size(Z.generators, 2);
Zs = Zonotope(SVector{n}(Z.center), SMatrix{n, p, Float64}(Z.generators));
Gs = Hyperplane(SVector{n}(G.a), G.b);
@btime is_intersection_empty($Zs, $Gs)

# new function
@btime _is_intersection_empty_v1($Z, $G)
@btime _is_intersection_empty_v1($Zs, $Gs)

# new function
@btime _is_intersection_empty_v2($Z, $G)
@btime _is_intersection_empty_v2($Zs, $Gs)

# final function
@btime _is_intersection_empty_v3($Z, $G)
@btime _is_intersection_empty_v3($Zs, $Gs)

  2.549 μs (50 allocations: 7.81 KiB)
  231.629 ns (0 allocations: 0 bytes)
  210.873 ns (1 allocation: 496 bytes)
  115.201 ns (0 allocations: 0 bytes)
  60.268 ns (0 allocations: 0 bytes)
  76.633 ns (0 allocations: 0 bytes)
  71.414 ns (0 allocations: 0 bytes)
  72.955 ns (0 allocations: 0 bytes)


false

sin `@simd`

```julia
  2.496 μs (50 allocations: 7.81 KiB)
  231.959 ns (0 allocations: 0 bytes)
  214.023 ns (1 allocation: 496 bytes)
  114.950 ns (0 allocations: 0 bytes)
  344.972 ns (0 allocations: 0 bytes)
  312.425 ns (0 allocations: 0 bytes)
```

con `@simd`


```julia
  2.524 μs (50 allocations: 7.81 KiB)
  233.880 ns (0 allocations: 0 bytes)
  218.264 ns (1 allocation: 496 bytes)
  110.941 ns (0 allocations: 0 bytes)
  343.959 ns (0 allocations: 0 bytes)
  317.128 ns (0 allocations: 0 bytes)
```

con `@avx`

```julia
  2.496 μs (50 allocations: 7.81 KiB)
  220.854 ns (0 allocations: 0 bytes)
  206.254 ns (1 allocation: 496 bytes)
  115.302 ns (0 allocations: 0 bytes)
  56.796 ns (0 allocations: 0 bytes)
  71.697 ns (0 allocations: 0 bytes)
```

In [34]:
using LoopVectorization

In [46]:
2.5e-6  / 243e-9

10.2880658436214